In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#Define path to tensor log files
logPath = "./tb_logs/"
#we define name_scope as tensorboard uses them

In [3]:
# Add summary statistics for visualization in Tensorboard
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [4]:
#Create input data which reads data from MNIST datasets. Perform on hot encoding to define the digit
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
#Using interactive session makes it to default session so we don not have to use sess
sess = tf.InteractiveSession()

In [6]:
#Define placeholders from MNISt data
with tf.name_scope('MNIST_Input'):
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y_")

In [7]:
#change MNIST data from a list of values to 28px X 28px X 1 grayscale cube
# which the convolution NN can use
with tf.name_scope("Input_Reshape"):
    x_image = tf.reshape(x, [-1,28,28,1], name="x_image")
    tf.summary.image('input_img', x_image, 5)

In [8]:
#Define helper functions to define bias and weights variables, convolution, and pooling layer
#we are using relu as our activation function. these must be utilized to a small positive number
# and with some noise so that we dont end up going to zero when comparing diffs

def weight_variables(shape, name=None):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variables(shape, name=None):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#Convolution and pooling - we do convolution and then pooling to control overfitting
def conv2d(x,W, name=None):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1],padding="SAME")

def max_pool_2x2(x, name=None):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [9]:
#define layers in NN

#1st convolution layer
#32 features of 5x5 patch of image
with tf.name_scope('Conv1'):
    with tf.name_scope('weights'):
        W_conv1 = weight_variables([5,5,1,32], name="weight")
        variable_summaries(W_conv1)
        
    with tf.name_scope('biases'):
        b_conv1 = bias_variables([32], name="bias")
        variable_summaries(b_conv1)
        
    #Do convolution on images, add bias and push through RELU activation
    conv1_wx_b = conv2d(x_image, W_conv1, name="conv2d") + b_conv1
    tf.summary.histogram('conv1_wx_b', conv1_wx_b)
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, name="conv2d") + b_conv1, name="relu")
    tf.summary.histogram("h_conv1", h_conv1)
    
    # take results and run through max_pool
    h_pool1 = max_pool_2x2(h_conv1, name="pool") #reduced to 14x14 image

In [10]:
#2st convolution layer
#Process the 32 features from convolution1 layer, in of 5x5 patch. Return 64 features weights and biases
with tf.name_scope('Conv2'):
    with tf.name_scope('weights'):
        W_conv2 = weight_variables([5,5,32,64], name="weight")
        variable_summaries(W_conv2)
       
    with tf.name_scope('biases'):
        b_conv2 = bias_variables([64], name="bias")
        variable_summaries(b_conv2)
        
    #Do convolution on output of 1st convolution layer. pool results
    conv2_wx_b = conv2d(h_pool1, W_conv2, name="conv2d") + b_conv2
    tf.summary.histogram('conv2_wx_b', conv2_wx_b)
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, name="conv2d") + b_conv2, name="relu")
    tf.summary.histogram("h_conv2", h_conv2)
    
    # take results and run through max_pool
    h_pool2 = max_pool_2x2(h_conv2, name="pool") #reduced to 7x7 image

In [11]:
#Fully connected layer
with tf.name_scope('FC'):
    W_fc1 = weight_variables([7*7*64,1024], name="weight")
    b_fc1 = bias_variables([1024], name="bias")

    # connect output of pooling layer 2 as input to fully connected layer
    h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1, name="relu")

In [12]:
# dropout some nuerons to reduce overfitting 
keep_prob = tf.placeholder(tf.float32, name="keep_prob")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [13]:
#Readout layer
with tf.name_scope('Readout'):
    W_fc2 = weight_variables([1024,10], name="weight")
    b_fc2 = bias_variables([10], name="bias")

In [14]:
#Define model
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#Loss measurement
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))

#loss optimzation
with tf.name_scope('loss_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


with tf.name_scope('accuracy'):
    #What is correct
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    #how accurate is it
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar("cross_entropy_sc1", cross_entropy)
tf.summary.scalar("training_accuracy", accuracy)

#TB-merge all
summarize_all = tf.summary.merge_all()
    
#initialize all the variables
sess.run(tf.global_variables_initializer())

In [15]:
#Write the data to visualize the computational graph
#TB - Write the default graph so that we can view it's structure
tbWriter = tf.summary.FileWriter(logPath, sess.graph)

In [16]:
#train the model
import time

In [17]:
#define nnumber of steps and how often we display progress
num_steps = 2000
display_every = 100

In [18]:
#Start timer
start_time = time.time()
end_time = time.time()

for i in range(num_steps):
    batch = mnist.train.next_batch(50)
    #train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob: 0.5})
    _, summary = sess.run([train_step, summarize_all], feed_dict={x:batch[0], y_:batch[1], keep_prob: 0.5})
    
    #periodic status display
    if i%display_every == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob: 1.0})
        end_time = time.time()
        print("step {0}, elapsed_time {1:.2f} seconds, training_accuracy {2:.3f}".format(i, end_time - start_time, train_accuracy * 100.0))
        
        tbWriter.add_summary(summary, i)


step 0, elapsed_time 0.73 seconds, training_accuracy 2.000
step 100, elapsed_time 46.89 seconds, training_accuracy 84.000
step 200, elapsed_time 92.35 seconds, training_accuracy 90.000
step 300, elapsed_time 137.50 seconds, training_accuracy 96.000
step 400, elapsed_time 182.96 seconds, training_accuracy 94.000
step 500, elapsed_time 229.46 seconds, training_accuracy 94.000
step 600, elapsed_time 274.88 seconds, training_accuracy 96.000
step 700, elapsed_time 327.64 seconds, training_accuracy 94.000
step 800, elapsed_time 388.00 seconds, training_accuracy 94.000
step 900, elapsed_time 451.59 seconds, training_accuracy 98.000
step 1000, elapsed_time 512.90 seconds, training_accuracy 96.000
step 1100, elapsed_time 577.93 seconds, training_accuracy 96.000
step 1200, elapsed_time 647.97 seconds, training_accuracy 96.000
step 1300, elapsed_time 727.82 seconds, training_accuracy 98.000
step 1400, elapsed_time 788.91 seconds, training_accuracy 100.000
step 1500, elapsed_time 847.91 seconds, t

In [19]:
#Display summary
#Time to train
end_time = time.time()
print("total training time for {0} batches: {1:.2f} seconds".format(i+1, end_time-start_time))

#Accuracy on test data
print("Test accuracy {0:.3f}%".format(accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0})*100.0))

sess.close()

total training time for 2000 batches: 1152.84 seconds
Test accuracy 97.840%


In [20]:
#To view tensorbaord
#run command
#tensorboard --log tb_logs